In [1]:
param_market = 'DE'

In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./item.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_item = {}
list_job = []
for i in range(len(input_)):
    if input_.loc[i, 'Vehicle_Page'] != '':
        vehicle_page = int(input_.loc[i, 'Vehicle_Page'])
        dict_item[input_.loc[i, 'Item_Number']] = {'vehicle_page': vehicle_page,
                                                   'crawler': 0,
                                                   'dict_vehicle': {},
                                                   'vehicle_1': '',
                                                   'vehicle_2': ''}
        for j in range(vehicle_page):
            list_job.append((i, j))

print('总数量：' + str(len(list_job)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(list_job)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                i, j = list_job[a]

                data = '{"scopedContext":{"catalogDetails":{"itemId":"' + input_.loc[i, 'Item_Number'] + '","categoryId":"' + input_.loc[i, 'Category_Id'] + '","marketplaceId":"' + input_.loc[i, 'Market_Id'] + '"}}}'

                # ua = UA.get_User_Agent_Requests()
                # ua['Content-Type'] = 'application/json'
                ua = {'Content-Type': 'application/json'}

                if param_market == 'US':
                    url = 'https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'DE':
                    url = 'https://www.ebay.de/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=DE&_stpos=10115&orig_cvip=true'
                elif param_market == 'AU':
                    url = 'https://www.ebay.com.au/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&ul=AU&_stpos=2019&orig_cvip=true'
                elif param_market == 'UK':
                    url = 'https://www.ebay.co.uk/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=GB&_stpos=BS81QU&orig_cvip=true'

                resp = requests.post(url,
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                list_header = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']
                
                # = = = = = = = = = = = = = = =
                
                col_engine = -1
                if param_market in ['US', 'AU', 'UK']:
                    name_make = 'Make'
                    name_model = 'Model'
                    name_engine = 'Engine'
                    name_year = 'Year'
                elif param_market == 'DE':
                    name_make = 'Marke'
                    name_model = 'Modell'
                    name_engine = 'Motor'
                    name_year = 'Baujahr'

                for k in range(len(list_header)):
                    if name_make in list_header[k]['textSpans'][0]['text']:
                        col_make = k
                    elif name_model in list_header[k]['textSpans'][0]['text']:
                        col_model = k
                    elif name_engine in list_header[k]['textSpans'][0]['text']:
                        col_engine = k
                    elif name_year in list_header[k]['textSpans'][0]['text']:
                        col_year = k
                        
                # = = = = = = = = = = = = = = =
                
                list_row = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']

                # = = = = = = = = = = = = = = =
                
                for row in list_row:
                    make = row['cells'][col_make]['textSpans'][0]['text'].strip()
                    model = row['cells'][col_model]['textSpans'][0]['text'].strip()
                    if col_engine == -1:
                        engine = ''
                    else:
                        engine = row['cells'][col_engine]['textSpans'][0]['text'].strip()
                    if param_market in ['US', 'AU', 'UK']:
                        list_year = [int(row['cells'][col_year]['textSpans'][0]['text'].strip())]
                    elif param_market == 'DE':
                        list_year = [int(year.split('/')[0].strip()) for year in row['cells'][col_year]['textSpans'][0]['text'].split('-')]

                    if make not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make] = {model: {engine: list_year}}
                    else:
                        if model not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model] = {engine: list_year}
                        else:
                            if engine not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = list_year
                            else:
                                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] += list_year

                    dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = sorted(list(set(dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine])))

                dict_item[input_.loc[i, 'Item_Number']]['crawler'] += 1

                # = = = = = = = = = = = = = = =
                
                if dict_item[input_.loc[i, 'Item_Number']]['vehicle_page'] == dict_item[input_.loc[i, 'Item_Number']]['crawler']:
                    list_vehicle = []
                    for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            list_year = []
                            for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                list_year += dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine]
                            list_year = list(set(list_year))
                            list_year.sort()

                            year_min = list_year[0]
                            year_max = list_year[-1]
                            if year_min == year_max:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                            else:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                    list_vehicle.sort()

                    dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'] = '\n'.join(list_vehicle)
                    
                    # = = = = = = = = = = = = = = =
                    
                    list_vehicle = []
                    for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                year_min = dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine][0]
                                year_max = dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine][-1]
                                if year_min == year_max:
                                    if engine == '':
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                    else:
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                                else:
                                    if engine == '':
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                                    else:
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                    list_vehicle.sort()

                    dict_item[input_.loc[i, 'Item_Number']]['vehicle_2'] = '\n'.join(list_vehicle)
                    
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                                             'Vehicle_1': dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'],
                                             'Vehicle_2': dict_item[input_.loc[i, 'Item_Number']]['vehicle_2']}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'

                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                                     'Vehicle_Page': input_.loc[i, 'Vehicle_Page'],
                                     'Category_Id': input_.loc[i, 'Category_Id'],
                                     'Market_Id': input_.loc[i, 'Market_Id']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[i, 'Item_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：5991

[ok] - 404888375340
[尝试次数：1] - [剩余数量：5961] - [当前时间：21:17:01]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：5960] - [当前时间：21:17:01]

[ok] - 315281836881
[尝试次数：1] - [剩余数量：5959] - [当前时间：21:17:01]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：5958] - [当前时间：21:17:01]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：5957] - [当前时间：21:17:01]

[ok] - 314966582400
[尝试次数：1] - [剩余数量：5956] - [当前时间：21:17:01]

[ok] - 404888375340
[尝试次数：1] - [剩余数量：5955] - [当前时间：21:17:01]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：5954] - [当前时间：21:17:01]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：5953] - [当前时间：21:17:01]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：5952] - [当前时间：21:17:01]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：5951] - [当前时间：21:17:01]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：5950] - [当前时间：21:17:01]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：5949] - [当前时间：21:17:01]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：5948] - [当前时间：21:17:01]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：5947] - [当前时间：21:17:01]

[ok] - 314966582305
[尝试次数：1] - [剩余数量：5946] - [当前时间：21:17:01]

[ok] - 386031543770
[尝试次数：4] - [剩余数量：5828] - [当前时间：21:17:22]

[ok] - 386031543770
[尝试次数：2] - [剩余数量：5827] - [当前时间：21:17:22]

[ok] - 404888353793
[尝试次数：11] - [剩余数量：5826] - [当前时间：21:17:22]

[ok] - 385219732846
[尝试次数：4] - [剩余数量：5825] - [当前时间：21:17:22]

[ok] - 385219732846
[尝试次数：5] - [剩余数量：5824] - [当前时间：21:17:23]

[ok] - 404888375340
[尝试次数：1] - [剩余数量：5823] - [当前时间：21:17:23]

[ok] - 404888353793
[尝试次数：14] - [剩余数量：5822] - [当前时间：21:17:23]

[ok] - 385219732846
[尝试次数：1] - [剩余数量：5821] - [当前时间：21:17:23]

[ok] - 404888353793
[尝试次数：13] - [剩余数量：5820] - [当前时间：21:17:23]

[ok] - 315253601277
[尝试次数：14] - [剩余数量：5819] - [当前时间：21:17:23]

[ok] - 385219732846
[尝试次数：1] - [剩余数量：5818] - [当前时间：21:17:24]

[ok] - 385219732846
[尝试次数：2] - [剩余数量：5817] - [当前时间：21:17:24]

[ok] - 404888353793
[尝试次数：18] - [剩余数量：5816] - [当前时间：21:17:24]

[ok] - 315301809894
[尝试次数：27] - [剩余数量：5815] - [当前时间：21:17:25]

[ok] - 314499477240
[尝试次数：4] - [剩余数量：5814] - [当前时间：21:17:25]

[ok] - 385219732846
[尝试次数：9] - [剩余数量：5813] - [当前时间：21:17:26]

[o

[ok] - 314499477240
[尝试次数：3] - [剩余数量：5696] - [当前时间：21:17:54]

[ok] - 404934834417
[尝试次数：1] - [剩余数量：5695] - [当前时间：21:17:55]

[ok] - 404934834417
[尝试次数：1] - [剩余数量：5694] - [当前时间：21:17:55]

[ok] - 404888353793
[尝试次数：9] - [剩余数量：5693] - [当前时间：21:17:55]

[ok] - 315301809894
[尝试次数：6] - [剩余数量：5692] - [当前时间：21:17:55]

[ok] - 404934834417
[尝试次数：1] - [剩余数量：5691] - [当前时间：21:17:55]

[ok] - 404934834417
[尝试次数：1] - [剩余数量：5690] - [当前时间：21:17:55]

[ok] - 404934834417
[尝试次数：1] - [剩余数量：5689] - [当前时间：21:17:56]

[ok] - 315253601277
[尝试次数：14] - [剩余数量：5688] - [当前时间：21:17:56]

[ok] - 404934834417
[尝试次数：1] - [剩余数量：5687] - [当前时间：21:17:56]

[ok] - 404581528701
[尝试次数：1] - [剩余数量：5686] - [当前时间：21:17:56]

[ok] - 404581528701
[尝试次数：1] - [剩余数量：5685] - [当前时间：21:17:57]

[ok] - 404581528701
[尝试次数：1] - [剩余数量：5684] - [当前时间：21:17:57]

[ok] - 315272078097
[尝试次数：1] - [剩余数量：5683] - [当前时间：21:17:57]

[ok] - 404581528701
[尝试次数：1] - [剩余数量：5682] - [当前时间：21:17:57]

[ok] - 404581528701
[尝试次数：1] - [剩余数量：5681] - [当前时间：21:17:58]

[ok] - 

[ok] - 402773738221
[尝试次数：4] - [剩余数量：5562] - [当前时间：21:18:24]

[ok] - 312770483848
[尝试次数：1] - [剩余数量：5561] - [当前时间：21:18:25]

[ok] - 312770483848
[尝试次数：2] - [剩余数量：5560] - [当前时间：21:18:25]

[ok] - 312770483848
[尝试次数：2] - [剩余数量：5559] - [当前时间：21:18:25]

[ok] - 312770483848
[尝试次数：2] - [剩余数量：5558] - [当前时间：21:18:25]

[ok] - 386860266062
[尝试次数：7] - [剩余数量：5557] - [当前时间：21:18:25]

[ok] - 404934834417
[尝试次数：5] - [剩余数量：5556] - [当前时间：21:18:26]

[ok] - 312770483848
[尝试次数：2] - [剩余数量：5555] - [当前时间：21:18:26]

[ok] - 312770483848
[尝试次数：4] - [剩余数量：5554] - [当前时间：21:18:26]

[ok] - 312770483848
[尝试次数：2] - [剩余数量：5553] - [当前时间：21:18:26]

[ok] - 312770483848
[尝试次数：2] - [剩余数量：5552] - [当前时间：21:18:26]

[ok] - 312770483848
[尝试次数：4] - [剩余数量：5551] - [当前时间：21:18:27]

[ok] - 315310778873
[尝试次数：3] - [剩余数量：5550] - [当前时间：21:18:27]

[ok] - 312770483848
[尝试次数：1] - [剩余数量：5549] - [当前时间：21:18:27]

[ok] - 384062841351
[尝试次数：1] - [剩余数量：5548] - [当前时间：21:18:27]

[ok] - 404934834417
[尝试次数：3] - [剩余数量：5547] - [当前时间：21:18:27]

[ok] - 3

[ok] - 404967869213
[尝试次数：2] - [剩余数量：5429] - [当前时间：21:18:53]

[ok] - 404967869213
[尝试次数：3] - [剩余数量：5428] - [当前时间：21:18:53]

[ok] - 404967869213
[尝试次数：1] - [剩余数量：5427] - [当前时间：21:18:54]

[ok] - 404967869213
[尝试次数：1] - [剩余数量：5426] - [当前时间：21:18:54]

[ok] - 404967869213
[尝试次数：1] - [剩余数量：5425] - [当前时间：21:18:54]

[ok] - 315333598148
[尝试次数：16] - [剩余数量：5424] - [当前时间：21:18:54]

[ok] - 386146598857
[尝试次数：3] - [剩余数量：5423] - [当前时间：21:18:54]

[ok] - 402770532070
[尝试次数：4] - [剩余数量：5422] - [当前时间：21:18:54]

[ok] - 404967869213
[尝试次数：1] - [剩余数量：5421] - [当前时间：21:18:55]

[ok] - 404967869213
[尝试次数：3] - [剩余数量：5420] - [当前时间：21:18:55]

[ok] - 404967869213
[尝试次数：1] - [剩余数量：5419] - [当前时间：21:18:55]

[ok] - 404967869213
[尝试次数：1] - [剩余数量：5418] - [当前时间：21:18:55]

[ok] - 386917055187
[尝试次数：4] - [剩余数量：5417] - [当前时间：21:18:55]

[ok] - 404967869213
[尝试次数：1] - [剩余数量：5416] - [当前时间：21:18:56]

[ok] - 404967869213
[尝试次数：1] - [剩余数量：5415] - [当前时间：21:18:56]

[ok] - 404967869213
[尝试次数：5] - [剩余数量：5414] - [当前时间：21:18:56]

[ok] - 

...

[ok] - 381564339136
[尝试次数：4] - [剩余数量：4497] - [当前时间：21:22:37]

[ok] - 386890213966
[尝试次数：1] - [剩余数量：4496] - [当前时间：21:22:37]

[ok] - 403883466091
[尝试次数：1] - [剩余数量：4495] - [当前时间：21:22:37]

[ok] - 403883466091
[尝试次数：1] - [剩余数量：4494] - [当前时间：21:22:38]

[ok] - 404960765075
[尝试次数：4] - [剩余数量：4493] - [当前时间：21:22:38]

[ok] - 403883466091
[尝试次数：1] - [剩余数量：4492] - [当前时间：21:22:38]

[ok] - 403883466091
[尝试次数：1] - [剩余数量：4491] - [当前时间：21:22:38]

[ok] - 403883466091
[尝试次数：1] - [剩余数量：4490] - [当前时间：21:22:39]

[ok] - 403883466091
[尝试次数：1] - [剩余数量：4489] - [当前时间：21:22:39]

[ok] - 403883466091
[尝试次数：1] - [剩余数量：4488] - [当前时间：21:22:39]

[ok] - 314887959037
[尝试次数：4] - [剩余数量：4487] - [当前时间：21:22:39]

[ok] - 404815443326
[尝试次数：3] - [剩余数量：4486] - [当前时间：21:22:39]

[ok] - 403883466091
[尝试次数：1] - [剩余数量：4485] - [当前时间：21:22:40]

[ok] - 403883466091
[尝试次数：1] - [剩余数量：4484] - [当前时间：21:22:40]

[ok] - 314221879888
[尝试次数：1] - [剩余数量：4483] - [当前时间：21:22:40]

[ok] - 315005996359
[尝试次数：2] - [剩余数量：4482] - [当前时间：21:22:41]

[ok] - 3

[ok] - 315362515579
[尝试次数：4] - [剩余数量：4364] - [当前时间：21:23:11]

[ok] - 315060410170
[尝试次数：4] - [剩余数量：4363] - [当前时间：21:23:11]

[ok] - 386625960913
[尝试次数：5] - [剩余数量：4362] - [当前时间：21:23:11]

[ok] - 315060410170
[尝试次数：5] - [剩余数量：4361] - [当前时间：21:23:11]

[ok] - 386625960913
[尝试次数：2] - [剩余数量：4360] - [当前时间：21:23:11]

[ok] - 315035970119
[尝试次数：1] - [剩余数量：4359] - [当前时间：21:23:11]

[ok] - 315324516179
[尝试次数：7] - [剩余数量：4358] - [当前时间：21:23:12]

[ok] - 386890213966
[尝试次数：6] - [剩余数量：4357] - [当前时间：21:23:12]

[ok] - 386892350455
[尝试次数：10] - [剩余数量：4356] - [当前时间：21:23:12]

[ok] - 315060410170
[尝试次数：3] - [剩余数量：4355] - [当前时间：21:23:12]

[ok] - 315281703979
[尝试次数：1] - [剩余数量：4354] - [当前时间：21:23:13]

[ok] - 386860273844
[尝试次数：8] - [剩余数量：4353] - [当前时间：21:23:13]

[ok] - 315035970119
[尝试次数：6] - [剩余数量：4352] - [当前时间：21:23:13]

[ok] - 315035970119
[尝试次数：6] - [剩余数量：4351] - [当前时间：21:23:13]

[ok] - 315035970119
[尝试次数：1] - [剩余数量：4350] - [当前时间：21:23:13]

[ok] - 404990564367
[尝试次数：1] - [剩余数量：4349] - [当前时间：21:23:14]

[ok] - 

[ok] - 386805837971
[尝试次数：1] - [剩余数量：4229] - [当前时间：21:23:45]

[ok] - 404745662885
[尝试次数：1] - [剩余数量：4228] - [当前时间：21:23:46]

[ok] - 386805837971
[尝试次数：1] - [剩余数量：4227] - [当前时间：21:23:46]

[ok] - 385840256475
[尝试次数：1] - [剩余数量：4226] - [当前时间：21:23:47]

[ok] - 315245518138
[尝试次数：2] - [剩余数量：4225] - [当前时间：21:23:48]

[ok] - 315299197697
[尝试次数：1] - [剩余数量：4224] - [当前时间：21:23:48]

[ok] - 404749150539
[尝试次数：1] - [剩余数量：4223] - [当前时间：21:23:49]

[ok] - 386625960913
[尝试次数：2] - [剩余数量：4222] - [当前时间：21:23:49]

[ok] - 404749150539
[尝试次数：1] - [剩余数量：4221] - [当前时间：21:23:49]

[ok] - 404866044907
[尝试次数：2] - [剩余数量：4220] - [当前时间：21:23:50]

[ok] - 404749150539
[尝试次数：1] - [剩余数量：4219] - [当前时间：21:23:50]

[ok] - 404749150539
[尝试次数：1] - [剩余数量：4218] - [当前时间：21:23:50]

[ok] - 404749150539
[尝试次数：1] - [剩余数量：4217] - [当前时间：21:23:51]

[ok] - 386945015273
[尝试次数：1] - [剩余数量：4216] - [当前时间：21:23:51]

[ok] - 404745662885
[尝试次数：8] - [剩余数量：4215] - [当前时间：21:23:52]

[ok] - 315405506676
[尝试次数：1] - [剩余数量：4214] - [当前时间：21:23:52]

[ok] - 3

[ok] - 386869134226
[尝试次数：2] - [剩余数量：4095] - [当前时间：21:24:11]

[ok] - 404008518975
[尝试次数：5] - [剩余数量：4094] - [当前时间：21:24:11]

[ok] - 386875866738
[尝试次数：1] - [剩余数量：4093] - [当前时间：21:24:12]

[ok] - 404899125515
[尝试次数：2] - [剩余数量：4092] - [当前时间：21:24:12]

[ok] - 405000463142
[尝试次数：1] - [剩余数量：4091] - [当前时间：21:24:12]

[ok] - 404899125515
[尝试次数：1] - [剩余数量：4090] - [当前时间：21:24:12]

[ok] - 404912785052
[尝试次数：1] - [剩余数量：4089] - [当前时间：21:24:12]

[ok] - 404917143385
[尝试次数：4] - [剩余数量：4088] - [当前时间：21:24:12]

[ok] - 404899125515
[尝试次数：5] - [剩余数量：4087] - [当前时间：21:24:13]

[ok] - 404953530773
[尝试次数：3] - [剩余数量：4086] - [当前时间：21:24:13]

[ok] - 386558278633
[尝试次数：3] - [剩余数量：4085] - [当前时间：21:24:13]

[ok] - 386766356909
[尝试次数：6] - [剩余数量：4084] - [当前时间：21:24:14]

[ok] - 386345807018
[尝试次数：2] - [剩余数量：4083] - [当前时间：21:24:15]

[ok] - 386345807018
[尝试次数：4] - [剩余数量：4082] - [当前时间：21:24:15]

[ok] - 386766356909
[尝试次数：10] - [剩余数量：4081] - [当前时间：21:24:15]

[ok] - 386345807018
[尝试次数：6] - [剩余数量：4080] - [当前时间：21:24:16]

[ok] - 

[ok] - 386271561546
[尝试次数：1] - [剩余数量：3961] - [当前时间：21:24:48]

[ok] - 386938821799
[尝试次数：2] - [剩余数量：3960] - [当前时间：21:24:48]

[ok] - 386271561546
[尝试次数：1] - [剩余数量：3959] - [当前时间：21:24:48]

[ok] - 386271561546
[尝试次数：1] - [剩余数量：3958] - [当前时间：21:24:48]

[ok] - 315249044971
[尝试次数：1] - [剩余数量：3957] - [当前时间：21:24:48]

[ok] - 386366108390
[尝试次数：1] - [剩余数量：3956] - [当前时间：21:24:49]

[ok] - 315249044971
[尝试次数：1] - [剩余数量：3955] - [当前时间：21:24:49]

[ok] - 315249044971
[尝试次数：1] - [剩余数量：3954] - [当前时间：21:24:49]

